In [1]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing import image
from kerastuner.tuners import RandomSearch

C:\Users\foreh\AppData\Local\Temp/ipykernel_16196/1121131459.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celebaRepo/FacialAttrCNN/celeba/greyscale"#archive/img_align_celeba/img_align_celeba"
images_data_size = 10000 

tf.keras.backend.clear_session()
width = 178
height = 218
x_ = 100                            # new value of x
image_size_ratio = width / height    # dimensions ratio
y_ = int(image_size_ratio * x_)  
model = Sequential()
model.add(tf.keras.Input(shape=(height, width,1)))
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=16,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=8, kernel_size=(3,3), padding='same'))
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=14, kernel_size=(3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=12,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=12,kernel_size=(2,2),padding = 'same'))
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Conv2D(filters=12,kernel_size=(3,3),padding = 'same'))
model.add(Conv2D(filters=10,kernel_size=(2,2),padding = 'same'))
## changed from (3,3)->(1,3)
model.add(tf.keras.layers.Normalization())

model.add(Conv2D(filters=12, kernel_size=(3, 3), padding='same'))
model.add(MaxPooling2D(pool_size=4))

model.add(Conv2D(filters=6, kernel_size=(3,3)))
model.add(Conv2D(filters=4, kernel_size=(2,2)))
model.add(tf.keras.layers.AveragePooling2D(pool_size=2))

model.add(tf.keras.layers.Normalization())

model.add(Flatten())
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(48))
model.add(Dense(16))
model.add(Dense(1, activation='sigmoid'))
original_image_size = (width,height)
df_org = pd.read_csv(attr_data_path)
print(df_org.columns)
for i in range(1, 100):
    if (i > 1):
        model = tf.keras.models.load_model('cnn85_checkpoints/iter'+str(i-1))
        print("model loaded: iter"+str(i-1))
        print("training model "+str(i))
    df = pd.DataFrame()
    if((i+1) % 20 == 0):
        df = df_org[(1+images_data_size*(i%20)):202600]
    else:
        df = df_org[(1+images_data_size*(i%20)):images_data_size*((i+1)%20)]
    df = df.replace(-1,0)
    #print(df.head(5))
    images_data = list()
    for idx in range(df.shape[0]):
        path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
        image = PIL.Image.open(path)
        image_array = np.asarray(image)
        images_data.append(image_array)
    images_data = np.array(images_data)
    def image_from_index(index, size=(original_image_size), path=images_data_path):
        path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
        # to read the image
        image = PIL.Image.open(path).resize(size)
        image_array = np.asarray(image)

        return image_array
    images = images_data # list of array of images
    labels = df[["image_id", "Male"]]  # dataframe of image features
    train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(i%6))
    y_test=test_labels.drop(["image_id"],axis=1)
    y_train=train_labels.drop(["image_id"],axis=1)

    X_test=test_images
    X_train=train_images
    #X_train = tf.expand_dims(X_train, axis=-1)
    print("Samples Training Set:",len(X_train))
    X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(i%5))
    print("Samples in Validation Data:",len(X_val))
    print("Samples in Test Data:",len(X_test))
    x_ = original_image_size[0]
    y_ = original_image_size[1]
    BATCH_SIZE = (8)
    print("Batch size: " + str(BATCH_SIZE))
    if i%3 == 1 or i%3 == 2:
          model.compile(optimizer = "nadam",loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    elif i%3 == 2 or i%3 == 0:
          print("SGD")
          opt = tf.keras.optimizers.SGD(learning_rate=0.3)
          model.compile(optimizer = opt,loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    else:
          print("ADAM")
          opt = tf.keras.optimizers.Adam(learning_rate=0.003)
          model.compile(optimizer = opt,loss="binary_crossentropy",metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 'binary_accuracy'])
    training_process = model.fit(X_train,y_train,epochs = 10,validation_data = (X_val,y_val),batch_size=BATCH_SIZE,shuffle = True)
    model.save('cnn85_checkpoints/iter'+str(i))
    print("Model " + str(i) +"evaluating:")
    model.evaluate(X_val,y_val)
    tf.keras.backend.clear_session()

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')
Samples Training Set: 6999
Samples in Validation Data: 1500
Samples in Test Data: 1500
Batch size: 8
Epoch 1/10
875/875 [==============================] - 117s 128ms/step - loss: 0.9957 - precision: 0.6310 - recall: 0.5774 - binary_accuracy: 0.6764 - val_loss: 0.4778 - val_precision: 0.8278 - val_recall: 0.602

In [ ]:
#attr_data_path = "list_attr_celeba.csv"
#images_data_path = "C:/Users/foreh/Documents/celeba/greyscale"

tf.keras.backend.clear_session()
width = 178
height = 218
df_org = pd.read_csv(attr_data_path)

df = df_org[190000:202600]
df = df.replace(-1,0)
#print(df.head(5))
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)
def image_from_index(index, size=(width,height), path=images_data_path):
    path = "{}/{}".format(str(path),str(df.iloc[index].image_id))
    # to read the image
    image = PIL.Image.open(path).resize(size)
    image_array = np.asarray(image)

    return image_array
images = images_data # list of array of images
labels = df[["image_id", "Male"]]  # dataframe of image features
labels=labels.drop(["image_id"],axis=1)



In [ ]:
for i in range(1, 40):
    tf.keras.backend.clear_session()
    print("###### EVAL MODEL "+str(i)+"######")
    model1 = tf.keras.models.load_model('cnn85_checkpoints/iter'+str(i))
    model1.evaluate(images,labels)

In [5]:
attr_data_path = "list_attr_celeba.csv"
images_data_path = "C:/Users/foreh/Documents/celebaRepo/FacialAttrCNN/celeba/greyscale"
images_data_size = 10000 

tf.keras.backend.clear_session()
width = 178
height = 218
x_ = 100                            # new value of x
image_size_ratio = width / height    # dimensions ratio
y_ = int(image_size_ratio * x_)
def initModel(hyperparam):
    model=Sequential()
    model.add(tf.keras.Input(shape=(height, width,1)))
    model.add(tf.keras.layers.Normalization())

    model.add(Conv2D(filters=16,
                     kernel_size=(3,3),
                     padding = 'same'))
    model.add(Conv2D(filters=8, 
                     kernel_size=(3,3), 
                     padding='same'))
    model.add(tf.keras.layers.Normalization())

    model.add(Conv2D(filters=14, 
                     kernel_size=(3, 3), 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=2)) # org: 2

    model.add(Dropout(0.2))

    model.add(Conv2D(filters=12,
                     kernel_size=(3,3),
                     padding = 'same'))
    model.add(Conv2D(filters=12,
                     kernel_size=(2,2),
                     padding = 'same'))
    model.add(tf.keras.layers.Normalization())

    model.add(Conv2D(filters=10, 
                     kernel_size=(3, 3), 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=2)) #org: 2

    model.add(Dropout(0.2))

    model.add(Conv2D(filters=12,
                     kernel_size=(3,3),
                     padding = 'same'))
    model.add(Conv2D(filters=10,
                     kernel_size=(2,2),
                     padding = 'same'))
    ## changed from (3,3)->(1,3)
    model.add(tf.keras.layers.Normalization())

    model.add(Conv2D(filters=12, 
                     kernel_size=(3, 3), 
                     padding='same'))
    #model.add(MaxPooling2D(pool_size=1)) #org: 4

    model.add(Conv2D(filters=6, 
                     kernel_size=(3,3)))
    model.add(Conv2D(filters=4, 
                     kernel_size=(2,2)))
    model.add(tf.keras.layers.AveragePooling2D(1)) #org: 2

    #model.add(tf.keras.layers.Normalization())

    model.add(Flatten())
    model.add(Dense(units=hyperparam.Int('dense_1', min_value=16, max_value=128, step=16)))
    model.add(Dense(units=hyperparam.Int('dense_2', min_value=8, max_value=32, step=8)))
    model.add(Dense(units=hyperparam.Int('dense_3', min_value=24, max_value=48, step=12)))
    model.add(Dense(units=hyperparam.Int('dense_4', min_value=12, max_value=20, step=4)))
    model.add(Dense(1, activation='relu'))
    opt = tf.keras.optimizers.SGD(learning_rate=hyperparam.Choice('learning_1',[1e-3, 1e-4, 1e-5]))
    model.compile(optimizer = "SGD",loss="binary_crossentropy",metrics = ['binary_accuracy'])
    return model
original_image_size = (width,height)
df_org = pd.read_csv(attr_data_path)
for i in range(1, 100):
    if (i > 1):
        model = tf.keras.models.load_model('cnn86_checkpoints/iter'+str(i-1))
        print("model loaded: iter"+str(i-1))
        print("training model "+str(i))
    df = pd.DataFrame()
    if((i+1) % 20 == 0):
        df = df_org[(1+images_data_size*(i%20)):202600]
    else:
        df = df_org[(1+images_data_size*(i%20)):images_data_size*((i+1)%20)]
    df = df.replace(-1,0)
    images_data = list()
    for idx in range(df.shape[0]):
        path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
        image = PIL.Image.open(path)
        image_array = np.asarray(image)
        images_data.append(image_array)
    images_data = np.array(images_data)
    images = images_data # list of array of images
    labels = df[["image_id", "Male"]]  # dataframe of image features
    train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(i%6))
    y_test=test_labels.drop(["image_id"],axis=1)
    y_train=train_labels.drop(["image_id"],axis=1)

    X_test=test_images
    X_train=train_images
    print("Samples Training Set:",len(X_train))
    X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(i%5))
    print("Samples in Validation Data:",len(X_val))
    print("Samples in Test Data:",len(X_test))
    BATCH_SIZE = (8)
    print("SGD")
    #training_process = model.fit(X_train,y_train,epochs = 10,validation_data = (X_val,y_val),batch_size=BATCH_SIZE,shuffle = True)
    tuner_search=RandomSearch(initModel,
                       objective="val_loss",
                       max_trials=500,directory='tuned85', project_name="anarchy")
    tuner_search.search(X_train,y_train,epochs=2,validation_data=(X_test,y_test))
    model.save('cnn86_checkpoints/iter'+str(i))
    print("Model " + str(i) +"evaluating:")

Trial 30 Complete [00h 03m 05s]
val_loss: 8.725140571594238

Best val_loss So Far: 6.2726945877075195
Total elapsed time: 01h 07m 43s

Search: Running Trial #31

Value             |Best Value So Far |Hyperparameter
64                |96                |dense_1
24                |32                |dense_2
48                |36                |dense_3
12                |12                |dense_4
1e-05             |0.001             |learning_1

Epoch 1/2
219/219 [==============================] - 93s 422ms/step - loss: 6.5585 - binary_accuracy: 0.5748 - val_loss: 6.2934 - val_binary_accuracy: 0.5920
Epoch 2/2
 48/219 [=====>........................] - ETA: 1:09 - loss: 6.9392 - binary_accuracy: 0.5501

KeyboardInterrupt: 